In [2]:
from flask import Flask, render_template, Response
import cv2
import time
import threading
from pygame import mixer  # Importing pygame for sound control

app = Flask(__name__)

# Load Haar cascades for face and eye detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye_tree_eyeglasses.xml')

# Timer variables
start_time = None
status_text = "Eyes Open"  # Default status
alarm_playing = False

# Initialize pygame mixer
mixer.init()
mixer.music.load("ssstwitter.com_1727753184570.mp3")  # Load the alarm sound


def detect_drowsiness():
    global start_time, status_text, alarm_playing

    cap = cv2.VideoCapture(0)
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Convert to grayscale for better detection
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces in the frame
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)
        eyes_detected = False  # Flag to track if eyes are detected

        for (x, y, w, h) in faces:
            # Draw rectangle around detected face
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

            # Region of interest for eye detection within the face
            roi_gray = gray[y:y + h, x:x + w]
            roi_color = frame[y:y + h, x:x + w]

            # Detect eyes within the face region
            eyes = eye_cascade.detectMultiScale(roi_gray, scaleFactor=1.1, minNeighbors=10, minSize=(30, 30))
            for (ex, ey, ew, eh) in eyes:
                cv2.rectangle(roi_color, (ex, ey), (ex + ew, ey + eh), (0, 255, 0), 2)
                eyes_detected = True  # Mark eyes as detected

        # Update status based on eye detection
        if eyes_detected:
            status_text = "Eyes Open"
            start_time = None  # Reset the timer

            # Stop the alarm if playing
            if alarm_playing:
                mixer.music.stop()
                alarm_playing = False
        else:
            if start_time is None:
                start_time = time.time()  # Start the timer if eyes are not detected
            elapsed_time = time.time() - start_time
            if elapsed_time > 1:  # Update status after 1 second
                status_text = "Eyes Closed"

                # Play the alarm if not already playing
                if not alarm_playing:
                    mixer.music.play(-1)  # Loop the alarm
                    alarm_playing = True

        # Display the status on the frame
        cv2.putText(frame, f"Status: {status_text}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Encode the frame into a byte stream
        _, buffer = cv2.imencode('.jpg', frame)
        frame = buffer.tobytes()

        # Yield the frame as a response
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

    cap.release()

@app.route('/')
def index():
    return render_template('driver.html')

@app.route('/video_feed')
def video_feed():
    return Response(detect_drowsiness(), mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [24/Nov/2024 12:08:36] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Nov/2024 12:08:36] "GET /static/styles.css HTTP/1.1" 404 -
127.0.0.1 - - [24/Nov/2024 12:08:37] "GET /video_feed HTTP/1.1" 200 -
